In [ ]:
import requests
import yaml
import os
from dotenv import load_dotenv
from IPython.display import display, Markdown, HTML
import json
from urllib.parse import urlparse, parse_qs, quote_plus
import datetime

load_dotenv()

API_KEY = os.getenv("TROVE_API_KEY")
headers = {"X-API-KEY": API_KEY}

In [47]:
display(Markdown("# Trove API v3 status"))
display(Markdown("This page runs the example queries listed in the [Trove API Console](https://troveconsole.herokuapp.com/v3/) and checks the results."))
display(Markdown(f"Last run: {datetime.datetime.now().isoformat()}"))

# Trove API v3 status

This page runs the example queries listed in the [Trove API Console](https://troveconsole.herokuapp.com/v3/) and tests the results.

Last run: 2023-06-14T15:11:04.412978

In [ ]:
response = requests.get("https://raw.githubusercontent.com/wragge/trove-api-console/master/examples-v3.yml")

In [ ]:
examples = yaml.safe_load(response.text)

In [ ]:
for section in examples["sections"]:
    display(Markdown(f"## {section['heading']}"))
    for example in section["examples"]:
        errors = False
        display(Markdown(f"### {example['heading']}"))
        display(Markdown(f"[{example['url']}](https://troveconsole.herokuapp.com/v3/?url={quote_plus(example['url'])})"))
        response = requests.get(example["url"], headers=headers)
        if response.status_code != 200:
            display(HTML(f"<div class='alert alert-block alert-danger'>❌ HTTP Error {response.status_code}</div>"))
            errors = True
        else:
            try:
                data = response.json()
            except json.decoder.JSONDecodeError:
                display(HTML(f"<div class='alert alert-block alert-danger'>❌ JSON Decode Error (the response doesn't seem to be in JSON)</div>"))
                errors = True
            else:
                if "category" in data and "id" not in data:
                    parsed_url = urlparse(example["url"])
                    params = parse_qs(parsed_url.query)
                    facets = False
                    if "facet" in params:
                        for cat in data["category"]:
                            if cat["facets"]:
                                facets = True
                                break
                        if facets is False:
                            display(HTML(f"<div class='alert alert-block alert-danger'>❌ No facet values</div>"))
                            errors = True
                    results = False
                    for cat in data["category"]:
                        if cat["records"]["total"] > 0:
                            results = True
                            break
                    if results is False:
                        display(HTML(f"<div class='alert alert-block alert-danger'>❌ No results</div>"))
                        errors = True
                elif "total" in data:
                    if data["total"] == 0:
                        display(HTML(f"<div class='alert alert-block alert-danger'>❌ No results</div>"))
                        errors = True
                elif "id" in data:
                    pass
                else:
                    display(HTML(f"<div class='alert alert-block alert-danger'>❌ Results format not recognised</div>"))
                    errors = True
        if errors is False:
            display(HTML(f"<div class='alert alert-block alert-success'>✅ Success!</div>"))
                